<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-02-05 13:02:29
-------------------
qualified stocks: 87
with latest results: 81
still star stocks: 45
-------------------
Initial Investment:  1.04 C
CY Investment:  1.17 C
Reserve:  1.23 L
Current:  1.16 C
-------------------
Today PnL: 1.54 L (1.35%)
Current PnL: -15.57 L (-13.26%)
CY Booked + Current PnL: -1.70 L (-1.45%)
-------------------
Total profit:  2.88 L
Total loss:  -18.45 L
-------------------
Total Booked + Current PnL: 11.41 L (10.94%)
Total Booked PnL: 26.98 L (25.87%)
Curr Year Booked PnL: 13.87 L (11.98%)
Prev Year Booked PnL: 13.11 L (12.57%)
Est FTT:  1.85 C
Est FTT PnL: 69.70 L (60.21%)
-------------------
Est LTT:  2.45 C
Est LTT PnL: 1.30 C (112.01%)
Deployed:  1.04 C
Current:  1.16 C
CAGR/XIRR %: 8.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
67        SBIN   760.30   863.00   -16.16       M-LC  65.52  203997.0   
35  HINDUNILVR  2430.78  2723.00   -12.65       X-LC  75.86  203024.0   
32     HAVELLS  1588.50  2077.12     3.12       X-LC  60.92  102470.0   
11    AXISBANK   986.10  1317.30   -29.20       H-LC  44.83  103601.0   
66     SBILIFE  1483.67  1928.65    -9.10       H-LC  78.16  100035.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
67       2518.0  24704.0        1.16          1.25  12.11  13.51  93.0   
35      -1162.0  25703.0       -1.02         -0.57  12.66  12.02  -3.0   
32       2394.0  28384.0       -2.03          2.39  27.70  30.76   NaN   
11       3019.0  30759.0        2.66          3.00  29.69  33.59   NaN   
66       -855.0  31111.0        0.76         -0.85  31.10  29.99   NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
67     0.10        1.78    20.37     XY25      NTT       BANKS  
35    -0.05        1.77    12.08     XY25      NTT        FMCG  
32     0.08        0.89    22.71      X40      ATH  ELECTRICAL  
11     0.10        0.90     7.13      X40      ATH       BANKS  
66    -0.03        0.87     9.88       AR      ATH   INSURANCE

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
19       CAMS  3643.00  5276.20     2.65       M-SC   1.15  105588.0   
83  UNIONBANK   123.87   163.00   -11.68       M-MC  86.21  135360.0   
6    ANGELONE  2328.67  3033.00   -14.85       H-SC  28.74  164538.0   
49    MEDANTA  1022.60  1486.00     1.81       X-SC  90.80  172452.0   
70    SIEMENS  6028.45  7969.85    -6.61       X-LC  12.64   94736.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
19       3584.0  42151.0       10.98          3.51  39.92  44.83  261.0   
83      -5480.0  49975.0        9.07         -3.89  36.92  31.59  -39.0   
6       13174.0  32611.0        8.84          8.70  19.82  30.25   77.0   
49      17017.0  53426.0        8.83         10.95  30.98  45.32    8.0   
70      -7748.0  40755.0        6.64         -7.56  43.02  32.20    NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
19     0.09        0.92    38.65       XR      ATH        MISC  
83    -0.11        1.18    17.58     XY24      NTT       BANKS  
6      0.40        1.44    23.92     X40N      NTT     FINANCE  
49     0.32        1.51    17.34     XY24      NTT  HEALTHCARE  
70    -0.19        0.83    32.43       AR      ATH  ELECTRICAL

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
68         SFL  1034.30  1287.00    11.15       H-SC  13.79  206790.0   
21   COFFEEDAY    59.14    80.00  -126.18       L-SC  20.69   47328.0   
26  EASEMYTRIP    18.68    26.40   -22.29       M-SC   2.30  117675.0   
27     FINEORG  4771.46  7168.71   -22.83       H-SC  51.72   73438.0   
14   BATAINDIA  1550.24  2096.00   -24.38       M-SC  48.28  113382.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
68     -45579.0  107241.0       -6.35        -18.06   51.86  24.43 -13.0   
21     -66221.0  106270.0       -5.01        -58.32  224.54  35.27   NaN   
26     -48726.0  117498.0       -4.69        -29.28   99.85  41.33 -91.0   
27      -7677.0   48432.0       -3.67         -9.46   65.95  50.24  -6.0   
14     -15288.0   60591.0       -3.48        -11.88   53.44  35.20  -8.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
68    -0.43        1.81     8.65     XY24      NTT       MISC  
21    -0.62        0.41     8.97       XR      NTT     HOTELS  
26    -0.41        1.03     0.00     XY24      NTT     TRAVEL  
27    -0.16        0.64     7.26     X40N      ATH  CHEMICALS  
14    -0.25        0.99    10.37      X40      NTT   FOOTWEAR

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
47      LUXIND  1448.19  2442.0    21.27       M-SC   8.05  152779.0   
28       GICRE   335.06   583.0     4.10       H-MC  35.63  178667.0   
19        CAMS  3643.00  5276.2     2.65       M-SC   1.15  105588.0   
4   ADANIPORTS  1103.69  1583.0    -9.12       M-LC  50.57  155290.0   
6     ANGELONE  2328.67  3033.0   -14.85       H-SC  28.74  164538.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
47      15201.0  79216.0        3.53         11.05  51.85  68.62    6.0   
28      32246.0  76112.0        5.28         22.02  42.60  74.00  109.0   
19       3584.0  42151.0       10.98          3.51  39.92  44.83  261.0   
4        4084.0  61588.0        4.48          2.70  39.66  43.43    NaN   
6       13174.0  32611.0        8.84          8.70  19.82  30.25   77.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
47     0.19        1.33    49.83     X40N      NTT   TEXTILES  
28     0.42        1.56    32.15     XY24      BTT  INSURANCE  
19     0.09        0.92    38.65       XR      ATH       MISC  
4      0.07        1.36     6.27     XY24      NTT       MISC  
6      0.40        1.44    23.92     X40N      NTT    FINANCE

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
19        CAMS  3643.00  5276.2     2.65       M-SC    1.15  105588.0   
13   BANKINDIA   112.90   190.0   -26.91       M-MC   95.40   95966.0   
1     AARTIIND   487.04   919.0     5.06       M-SC  100.00   77029.0   
72    SONACOMS   578.05  1006.0   -31.00       M-MC   17.24   89740.0   
62  SAMMAANCAP   170.35   326.0  -173.72       M-SC   24.14   86196.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
19       3584.0   42151.0       10.98          3.51   39.92  44.83  261.0   
13      -4289.0   72752.0        3.99         -4.28   75.81  68.29  -12.0   
1       -4307.0   76444.0        1.91         -5.29   99.24  88.69   -8.0   
72     -11419.0   86312.0        5.62        -11.29   96.18  74.03   -8.0   
62     -16014.0  109400.0        4.38        -15.67  126.92  91.37    NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
19     0.09        0.92    38.65       XR      ATH       MISC  
13    -0.06        0.84    19.61       XR      NTT      BANKS  
1     -0.06        0.67    15.94       XR      NTT  CHEMICALS  
72    -0.13        0.78     5.62       AR      BTT       AUTO  
62    -0.15        0.75     7.89       XR      NTT    FINANCE

In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
29  GREENPANEL   390.01   537.00    87.45       M-SC  88.51  164331.0   
28       GICRE   335.06   583.00     4.10       H-MC  35.63  178667.0   
46        LTIM  5664.39  7262.39    15.42       X-LC  91.95  182475.0   
64     SBICARD   765.91  1075.00   -21.56       H-MC  98.85  192641.0   
39        INFY  1436.24  2275.00     6.64       X-LC  56.32  198240.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%   XIRR  \
29      -8443.0  73555.0       -1.38         -4.89  44.76  37.69  -17.0   
28      32246.0  76112.0        5.28         22.02  42.60  74.00  109.0   
46      12543.0  35400.0        5.04          7.38  19.40  28.21    7.0   
64      12652.0  59988.0       -0.67          7.03  31.14  40.36   -1.0   
39      48871.0  38359.0        2.30         32.72  19.35  58.40   33.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
29    -0.11        1.43    29.59     XY24      NTT       MISC  
28     0.42        1.56    32.15     XY24      BTT  INSURANCE  
46     0.35        1.59    34.04     X200      ATH         IT  
64     0.21        1.68    23.48     XY24      NTT    FINANCE  
39     1.27        1.73    38.55      X40      BTT         IT

In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction     RSP   Current  \
1     AARTIIND   487.04   919.0     5.06       M-SC  100.00   77029.0   
64     SBICARD   765.91  1075.0   -21.56       H-MC   98.85  192641.0   
38  INDUSINDBK  1354.50  1800.0   -29.02       H-MC   97.70   60477.0   
82  UJJIVANSFB    59.50    60.0   -31.87       M-SC   96.55   63036.0   
13   BANKINDIA   112.90   190.0   -26.91       M-MC   95.40   95966.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
1       -4307.0  76444.0        1.91         -5.29  99.24  88.69  -8.0   
64      12652.0  59988.0       -0.67          7.03  31.14  40.36  -1.0   
38     -16730.0  42122.0        4.77        -21.67  69.65  32.89 -21.0   
82     -38114.0  38963.0       -1.07        -37.68  61.81   0.84 -27.0   
13      -4289.0  72752.0        3.99         -4.28  75.81  68.29 -12.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
1     -0.06        0.67    15.94       XR      NTT  CHEMICALS  
64     0.21        1.68    23.48     XY24      NTT    FINANCE  
38    -0.40        0.53    14.52       XR      NTT      BANKS  
82    -0.98        0.55    15.95     X40N      NTT      BANKS  
13    -0.06        0.84    19.61       XR      NTT      BANKS

In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.48
1     25       42.49
2     50       73.13

In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    41.90
X40     14.56
X40N    12.73
AR      10.75
XR       9.99
XY25     7.10
X200     1.59
SR       1.36
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
M-SC    24.30
X-LC    21.10
H-SC    16.80
M-MC     7.97
H-MC     6.21
X-MC     6.19
M-LC     5.43
X-SC     4.63
H-LC     3.21
L-SC     2.47
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
BANKS               14.49        -11.64   62.89
FINANCE             10.80        -17.49   65.82
FMCG                10.41         -2.03   32.96
IT                   8.74        -11.19   66.60
MISC                 7.90        -11.57   50.39
PAINTS               5.96        -20.36   55.98
INSURANCE            5.16          1.08   48.20
CHEMICALS            4.09        -15.37   91.39
AUTO                 3.11        -15.60   66.43
FOOTWEAR             2.97        -21.12   61.14
HEALTHCARE           2.83         -4.05   39.88
ELECTRICAL           2.72         -5.90   41.59
POWER                2.19        -12.72   57.81
TRAVEL               2.11        -27.24   79.19
TYRES                2.01          1.10   30.78
TEXTILES             2.00          0.00   52.52
CERAMICS             1.91        -30.01   82.90
REFINERIES           1.77         -0.60   19.14
STEEL                1.63        -20.58  110.45
JEWELLERY            1.46        -72.81  127.45
ENGINEERING          1.32        -20.66   67.14
CEMENT               1.27        -29.57   96.23
ENTERTAINMENT        1.06        -34.43  102.03
AC                   0.84        -32.23  105.22
DURABLES             0.82         -7.62   60.78
HOTELS               0.41       -139.92  224.54

In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3075930.0     31
XR        1093935.0     14
X40N      1007018.0     15
AR         852819.0     10
X40        580248.0     10
SR         189347.0      2
XY25       135787.0      4
X200        35400.0      1

In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
M-SC        2017757.0     25
H-SC        1812706.0     15
M-MC         689987.0      7
X-LC         668562.0     13
X-SC         373867.0      5
H-MC         344481.0      5
L-SC         314213.0      4
X-MC         270854.0      5
M-LC         233782.0      4
H-LC         132670.0      3

In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
M-SC       XY24      976260.0     10
H-SC       XY24      757603.0      5
M-MC       XY24      530923.0      5
H-SC       X40N      459620.0      6
M-SC       XR        416840.0      6
X-LC       X40       365868.0      6
M-SC       X40N      357973.0      5
H-SC       AR        313793.0      1
H-MC       XY24      302359.0      4
H-SC       XR        281690.0      3
M-LC       XY24      209078.0      3
L-SC       XR        172271.0      2
X-LC       AR        156211.0      3
M-SC       AR        136514.0      2
X-SC       SR        119768.0      1
X-LC       XY25      111083.0      3
X-SC       XR        108260.0      1
H-LC       X40       101559.0      2
X-MC       X40N       97012.0      2
X-SC       X40N       92413.0      2
M-MC       AR         86312.0      1
L-SC       XY24       83909.0      1
M-MC       XR         72752.0      1
X-MC       AR         70845.0      1
M-SC       SR         69579.0      1
           X40        60591.0      1
L-SC       AR         58033.0      1
X-SC       XY24       53426.0      1
X-MC       X40        52230.0      1
           XY24       50767.0      1
H-MC       XR         42122.0      1
X-LC       X200       35400.0      1
H-LC       AR         31111.0      1
M-LC       XY25       24704.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
